In [1]:
from flask import Flask, request, redirect, url_for,send_from_directory, render_template_string
from werkzeug.utils import secure_filename
import caffe
import json
import numpy as np
import os
from sklearn import svm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import struct
import 

Train: 2182 Test:243
Ratio: 9:1

In [2]:
deployPrototxt = "/opt/swm-ml-docker/swm-ml-master/bvlc_reference_caffenet/deploy.prototxt"
caffeModel = "/opt/swm-ml-docker/swm-ml-master/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel"
featureVectorDirectory = "/opt/swm-ml-docker/data/train"
metadataDirectory = '/opt/swm-ml-docker/metadata_bak/'
imageMeanFile = '/opt/swm-ml-docker/ilsvrc_2012_mean.npy'
layerName = 'fc7'

In [3]:
print("Loading model ...")
cnn = caffe.Net(deployPrototxt, caffe.TEST, weights=caffeModel)
print("Done loading")

Loading model ...
Done loading


In [4]:
# reshape to accept one image at a time.
(_, c, w, h) = cnn.blobs['data'].data.shape
cnn.blobs['data'].reshape(1, c, w, h)
transformer = caffe.io.Transformer({'data': cnn.blobs['data'].data.shape})
transformer.set_mean('data', np.load(imageMeanFile).mean(1).mean(1))
transformer.set_transpose('data', (2,0,1))
transformer.set_raw_scale('data', 255.0)

In [5]:
def loadFeatureVectors(dirPath, d):
    """
    Loads all feature vectors from the local directory into a single nxd matrix
    where n is the number of feature vectors and the d is the dimensionality.
    Returns the matrix and a list of file names in one-to-one correspondence
    with the rows of the matrix.
    """
    allFiles = os.listdir(dirPath)
    data = np.zeros([len(allFiles), d])
    for i, fileName in enumerate(allFiles):
        with open("%s/%s" % (dirPath, fileName), "rb") as inputFile:
            data[i] = struct.unpack('f'*d, inputFile.read())
    return (data, allFiles)

def loadMetaData(metadataDir):
    metadataByImageHash = {}
    allFiles = os.listdir(metadataDir)
    for fileName in allFiles:
        with open("%s/%s" % (metadataDir, fileName), "r") as jsonFile:
            metadata = json.load(jsonFile)
            metadataByImageHash[metadata['Image Hash']] = metadata
    return metadataByImageHash

# Simple case
def makeBinaryLabels(metadataByImageHash, allImageHashes):
    labels = []
    for imageHash in allImageHashes:
        metadata = metadataByImageHash[imageHash]
        labels.append(1 if (metadata['Secondary'] == "NA") else 0)
    return labels

In [6]:
print("Training SVM")
data, allImageHashes = loadFeatureVectors(featureVectorDirectory, d = 4096)
metadataByImageHash = loadMetaData(metadataDirectory)
labels = makeBinaryLabels(metadataByImageHash, allImageHashes)
clf = svm.SVC(probability=True)
clf.fit(data, labels)
print("Finished training ready to classify images")

Training SVM
Finished training ready to classify images


In [7]:
featureVectorTestDirectory = "/opt/swm-ml-docker/data/test"
dataTest, allImageHashesTest = loadFeatureVectors(featureVectorTestDirectory, d = 4096)
labelsTest = makeBinaryLabels(metadataByImageHash, allImageHashesTest)

In [8]:
clf.score(dataTest,labelsTest)

0.80246913580246915

In [9]:
prediction=clf.predict(dataTest)

In [10]:
CM = metrics.confusion_matrix(labelsTest,prediction)

NameError: name 'metrics' is not defined

In [ ]:
print CM

In [ ]:
fpr,tpr,thesholds=metrics.roc_curve(labelsTest,prediction)

In [ ]:
%%bash
caffe --version

In [ ]:
transpose(CM)

In [17]:
i=0
file=open("FP.txt",'w')
while i<len(prediction):
    if prediction[i]==1 and labelsTest[i]==0:
        file.write(str(i))
        file.write('\n')
    i=i+1
file.close()        
        

In [51]:
filepath1="/opt/swm-ml-docker/data/test.txt"
filepath2="/opt/swm-ml-docker/data/FP.txt"
file=open("FP_imageName.txt",'w')
l=[]
with open(filepath2,'r') as fp:
    for line in fp:
        l.append(int(line))

i=0
FPImageHashes=[]
with open(filepath1,'r') as fp:
    for line in fp:
        if i in l:
            line = line.strip('\n')
            file.write(line+".jpg")
            file.write('\n')
            FPImageHashes.append(line)
        i=i+1
file.close()               
       

In [52]:
imageName=[]
file=open("imageName.txt",'w')
for i in FPImageHashes:
    metadata=metadataByImageHash[i]
    file.write(metadata["Image Name"])
    file.write('\n')
    GoogleIds.append(metadata["Image Name"])
file.close()